# Azure monitor resources basics

Within the Azure ecosystem there are quite a few components that can be used to monitor applications and services. These components are part of the Azure Monitor stack, which is a set of services and tools that provide monitoring, diagnostics, and alerting capabilities for Azure resources and applications.

![Azure Monitor](.files/azure_monitor.png)

Within Azure monitor we have the ability query logs, create alerts and build dashboards. These features are all available in the azure portal, but also using the Azure CLI or Bicep templates.

All the services we need in azure are defined via Bicep, but can also be deployed in a Azure portal UI, or via az cli.


For the workshop today we will be using the following resources:
- **Log Analytics workspace**: This is a service within Azure Monitor that allows you to collect and analyze log data from various sources. It provides a powerful query language (Kusto Query Language) for analyzing log data and creating custom dashboards and alerts.
- **Application Insights**: This is a service within Azure Monitor that provides application performance monitoring and diagnostics. It collects telemetry data from your applications, including request rates, response times, and failure rates, and provides insights into the performance and usage of your applications.


Let's try to deploy these resources, you can choose to either do it via the Azure portal or using Bicep templates. Follow the steps below if you don't have the Azure CLI installed yet.
<details>
  <summary>How to install the Azure CLI</summary>

**Windows**
```Powershell
winget install --exact --id Microsoft.AzureCLI
```

**Ubuntu/Linux with APT**
```bash
sudo apt-get update && sudo apt-get install azure-cli -y
```

**MacOS with Homebrew**
```bash
brew update && brew install azure-cli
```
</details>

Using the azcli, you can log in to your Azure account using the following command:

```bash
az login
```

Then select the subscription you want to use for this workshop:

```bash	
az account set --subscription "YOUR_SUBSCRIPTION_OID"
```

Let's check if everything is working as expected by creating a resource group in the swedencentral region that we'll use for this workshop.

```bash
az group create --name "rg-observabilityworkshop-dev" --location 'swedencentral'
```

**Deploy Log Analytics workspace and Application Insights**

Let's create a Log Analytics workspace and Application Insights resource in the resource group we just created. You can use either the azcli and the following commands, The Bicep files in the 01 folder or using the portal. however it is recommended to use the Bicep files to as we will reuse them in the nest sections.

<details>
  <summary>Using the Azure CLI</summary>

```powershell

$UserInitials = "ABCD" # Replace with your initials

$Timestamp = Get-Date -Format "yyMMddHHmmss"

# Microsoft Observability Workshop - msows
$SystemName = "msows${UserInitials}"
$Environment = "dev"
$Location = "swedencentral"

$ResourceGroupName = "rg-observabilityworkshop-dev"
$workspacename = "log-${SystemName}-dev"
$appiName = "appi-${SystemName}-dev"
az monitor log-analytics workspace create --name $workspacename --resource-group $ResourceGroupName --location 'swedencentral'
az monitor app-insights component create --app $appiName --workspace $workspacename --location 'swedencentral' --kind other -g $ResourceGroupName --application-type other --retention-time 120
```
</details>

<details>
  <summary>Using Bicep</summary>

There is a Deploy.ps1 file in the 01 folder that you can use to deploy the resources, that is nearly identical to this code

```powershell
$UserInitials  = "ABCD" # Replace with your initials

$Timestamp = Get-Date -Format "yyMMddHHmmss"

# Microsoft Observability Workshop - msows
$SystemName = "msows${UserInitials}"
$Environment = "dev"
$Location = "swedencentral"

$ResourceGroupName = "rg-observabilityworkshop-dev"

az deployment group create `
    --name "ObservabilityWorkshop${Timestamp}" `
    --mode "Complete" `
    --resource-group $ResourceGroupName `
    --template-file ./01/main.bicep `
    --parameters `
        env=$Environment `
        systemName=$SystemName `
        location=$Location `
        deploymentSuffix=$Timestamp
```
</details>

Good, you should now have the following resources in your resource group:
- **Log Analytics workspace**
- **Application Insights**

Now that we've deployed our azure resources, let's take a look at them, and what functionality they provide out of the box.

**Log Analytics workspace**

Within the Log analytics workspace we have a few different features that we can use to collect, query and analyze the raw logs, I'll just 4 of them that you might find useful:

* **Logs** From the logs blade we can query the logs using Kusto Query Language (KQL). You will generally use this to investigate issues deeper, or write queries that you can use in dashboards or for alerts. The logs blade is also where you can create custom queries and save them for later use.
* **Tables** Overview of what tables are available in the workspace, and the ability to update retention periods. This is a good place to start if you want to see what data is available for querying.
* **Agents** Agents are application that you can run on windows or linux machines to collect logs and send them to the Log Analytics workspace. This is a good way to collect logs from on-premises machines, or from virtual machines that are not running in Azure. They can also serve as proxies machines that can't access the internet directly.
* **Data Export rules** Data export rules are used to export data from the Log Analytics workspace to other Azure services, such as Azure Storage or Event Hubs. This is a good way to archive logs, or to send logs to other services for further processing.

**Application Insights**
Application Insights is a service within Azure Monitor that provides application performance monitoring and diagnostics. It receives telemetry data from your applications, and provides a few more features to display your logs, and investigate how they are collected.

**Application map**: The application map is a visual representation of the components in your application and how they are connected. It shows the dependencies between components, and visualizes the latencies, number of requests, and failure rates for each component. this is great for a quick overview of your entire platform.

**Live metrics**: Operational metrics about your web servers, such as CPU and memory usage, request rates, and response times. and dependency calls. 

**Transaction search**
The transaction search is a powerful tool that allows you to search for specific requests and dependencies in your application. You can filter by time range, request URL, response code, and other parameters. and investigate the request end to end to debug issues. This is a great tool for investigating issues in production, as it allows you to see the entire request flow and identify where the issue occurred.

<img src=".files/appi_transaction_search_query.png" width="600">

you can pres on any log entry to show the end to end transaction, and see the entire request flow, as well as all custom dimensions of the log entries.

<img src=".files/appi_transaction_search_endtoend.png" width="600">


**Availability**: The Availability feature in Application insight allow you to create health check probes via HTTP(s) so see if your applications is healthy and responding, if the certificate is about the expire, and optionally create alerts based on this.


<img src=".files/appi_Availability.png" width="600">

**Failure analysis**:
Failure analysis gives you quick insight into errors, exceptions and failed requests in your application. It shows you the number of failures, the failure rate, and the most common failure types. This is a great tool for identifying issues in your application and troubleshooting them.


**Performance analysis**: 
The performance analysis feature in Application Insights allows you to analyze the response times of your application and identify performance bottlenecks. It shows you the average response time, the number of requests, and the most common response times. This is a great tool for identifying performance issues in your application and troubleshooting them. eg if you have a slow dependency call, or if your have poorly optimized queries in your application.

<img src=".files/appi_preformance_analysis.png" width="600">


#### Next steps
We can now take a look at collecting telemetry from azure services, Jump to the next section.

